<a href="https://colab.research.google.com/github/reedhodges/pytorch-loop-integrals/blob/main/pytorch_loop_integrals.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Using PyTorch to classify loop integrals by the type of their divergence

Before starting, run all the cells in the 'Preliminaries' section to make sure you've downloaded all the necessary files.

In [2]:
import torch
import torchvision

print(torch.__version__)
print(torchvision.__version__)

device = "cuda" if torch.cuda.is_available() else "cpu"

print(f"Device: {device}")

2.3.0.dev20240314
0.18.0.dev20240314
Device: cpu


#### Optional: using GPU-accelerated PyTorch on Apple Silicon Macs

If you have an Apple Silicon Mac, you can use GPU-accelerated PyTorch, but you need to install the Preview (Nightly) version of PyTorch.  You can set up a Python virtual environment with the following in the command line:

```zsh
python -m venv pytorch-nightly
```

Activate the virtual environment with:

```zsh
source pytorch-nightly/bin/activate
```

Then install the Preview (Nightly) version of PyTorch, followed by whatever other packages are necessary.

```zsh
pip install --pre torch torchvision torchaudio --index-url https://download.pytorch.org/whl/nightly/cpu
```

Make sure the Jupyter kernel is set to use this virtual environment.  If you have successfully installed the Nightly build, the PyTorch version should have `dev` in it.

In [1]:
import torch
import torchvision

print(torch.__version__)
print(torchvision.__version__)

2.3.0.dev20240314
0.18.0.dev20240314


If everything works, the following should output a tensor with a single one that is stored on the `mps` device.

In [2]:
if torch.backends.mps.is_available():
    mps_device = torch.device("mps")
    x = torch.ones(1, device=mps_device)
    print (x)
else:
    print ("MPS device not found.")

tensor([1.], device='mps:0')


Let's set the device to "mps" for our training later.

In [3]:
import torch

device = "mps" if torch.backends.mps.is_available() else print("MPS device not found.")
print(f"Device: {device}")

Device: mps


### Preliminaries

In [4]:
import requests
import zipfile
from pathlib import Path

path_to_data = Path("data/")

if path_to_data.is_dir():
    print(f"[INFO] Directory {path_to_data} already exists, skipping download.")
else:
    print(f"[INFO] Creating {path_to_data} directory...")
    path_to_data.mkdir(parents=True, exist_ok=True)
    with open("integrand_data.zip", "wb") as f:
        url = "https://github.com/reedhodges/pytorch-loop-integrals/raw/main/integrand_data.zip"
        response = requests.get(url)
        print(f"[INFO] Downloading zip from {url}...")
        f.write(response.content)

    with zipfile.ZipFile("integrand_data.zip", "r") as zip_ref:
        print(f"[INFO] Extracting zip...")
        zip_ref.extractall()

files_to_download = [
    {"path": "engine.py", "url": "https://raw.githubusercontent.com/reedhodges/pytorch-loop-integrals/main/engine.py"},
    {"path": "utils.py", "url": "https://raw.githubusercontent.com/reedhodges/pytorch-loop-integrals/main/utils.py"}
]

for file_info in files_to_download:
    file_path = file_info["path"]
    file_url = file_info["url"]
    if not Path(file_path).is_file():
        print(f"[INFO] Downloading {file_path}...")
        with open(file_path, "wb") as f:
            response = requests.get(file_url)
            f.write(response.content)
    else:
        print(f"[INFO] File {file_path} already exists, skipping download.")

print(f"[INFO] Done!")

[INFO] Directory data already exists, skipping download.
[INFO] File engine.py already exists, skipping download.
[INFO] File utils.py already exists, skipping download.
[INFO] Done!


### Set up data and model

In [5]:
from utils import fix_error_with_weights_download
from torchvision.models import efficientnet_b0, EfficientNet_B0_Weights

image_path_list = list(path_to_data.glob("*/*/*.png"))

# this is a hack to fix the weights download issue
fix_error_with_weights_download()

weights = EfficientNet_B0_Weights.DEFAULT
model = efficientnet_b0(weights=weights).to(device)

data_transform = weights.transforms()

for param in model.features.parameters():
    param.requires_grad = False

### Train

To use experiment tracking, you'll need a [Weights & Biases account](https://wandb.ai/site), which is free for personal use.  

In [ ]:
import wandb

wandb.login()

In [6]:
import os
from torch import nn, optim
from engine import create_data_loaders, train

BATCH_SIZE = 32
NUM_WORKERS = os.cpu_count()
NUM_EPOCHS = 3

train_loader, test_loader, class_names, class_dict = create_data_loaders(path_to_data, batch_size=BATCH_SIZE, num_workers=NUM_WORKERS, train_transform=data_transform, test_transform=data_transform)

model.classifier = nn.Sequential(
    nn.Dropout(p=0.2, inplace=True),
    nn.Linear(in_features=1280,
              out_features=len(class_names)).to(device)
)

loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

wandb.init(project="pytorch-loop-integrals",
           config={"batch_size": BATCH_SIZE,
                   "num_workers": NUM_WORKERS,
                   "num_epochs": NUM_EPOCHS,
                   "Loss Function": str(loss_fn),
                   "Optimizer": str(optimizer)})
 
train(device, model, train_loader, test_loader, loss_fn, optimizer, epochs=NUM_EPOCHS)

 33%|███▎      | 1/3 [01:26<02:52, 86.11s/it]


------------------------------

Epoch 1
Train Loss: 0.7053, Train Accuracy: 70.75
Test Loss:  0.7493, Test Accuracy:  76.50


 67%|██████▋   | 2/3 [02:50<01:25, 85.29s/it]


------------------------------

Epoch 2
Train Loss: 0.5768, Train Accuracy: 77.25
Test Loss:  0.5434, Test Accuracy:  79.50


100%|██████████| 3/3 [04:16<00:00, 85.54s/it]


------------------------------

Epoch 3
Train Loss: 0.5202, Train Accuracy: 79.38
Test Loss:  0.5839, Test Accuracy:  78.00
